In [1]:
import _dunsros as DR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tqdm import tqdm

from kan import *
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [60]:
# условно константы
k_og = 178                      #газовый фактор м.**3/ст.м.**3
bo = 1.197                      #объемный коэффициент нефти
bw = 1                          #объемный коэффициент воды
bg = 9.1 * 1e-3                 #объемный коэффициент газа
rhow = 1 * 1e3                  #плотность воды
mug = 0.016 * 1e-3              #вязкость газа

# условно переменные
T = 50                          #температура гр.Ц.
d = 0.154                       #диаметр трубы
theta = 90                      #угол наклона скважины
q0 = 1000                       #дебит нефти н.у.
qg0 = 1 * 1e3                   #дебит газа н.у.
rs = 5                          #газовый фактор — коэффициент растворимости газа в нефти (конденсации) или коэффициент выделения газа из нефти (выкипания)
rho0 = 800                      #плотность нефти
rhog = 700                      #плотность газа
muo = 0.97 * 1e-3               #вязкость нефти
epsilon = 18.288 * 1e-6         #шероховатость стенки трубы
sigmao = 8.41 * 1e-3            #коэффициент поверхностного натяжения нефти
# wc = 1 - qg0 / q0 / k_og        #коэффициент обводненности
wc = 0.009
p = 100                         #давление (где?)



# расчетные величины (кроме обводненности, которую удобно использовать как входную переменную)
qo = q0 * (1 - wc) * bo                                 #объем нефти в пластовых условиях
ql = ((1 - wc) * bo + wc * bw) * qo                     #объем жидкости в пластовых условиях
qg = (qg0 - q0 * rs * (1 - wc)) * bg                    #объем газа в пластовых условиях
rho_lrc = rho0 * (1 - wc) / bo + rhow * wc / bw         #плотность жидкой фазы
rho_grc = rhog / rs                                     #плотность газа

delta_qg = 1000

sec_p_day = 24 * 60 * 60                                #переведем все в систему СИ
ql = ql / sec_p_day
qg = (qo * rs * bg * (1 - wc) + delta_qg)  / sec_p_day

args_params = {
        "d": d,                         #диаметр в метрах
        "theta_deg": theta,             #угол
        "ql_rc_m3day": ql,              #дебит жидкости
        "qg_rc_m3day": qg,              #расход газа
        "rho_lrc_kgm3": rho_lrc,        #плотность жидкой фазы
        "rho_grc_kgm3": rho_grc,        #плотность газа
        "p": p,                         #давление (где?)
        "T" : T,                        #температура смеси
        "wc": wc,                       #коэффициент заводнения
        "sigmao": sigmao,               #коэффициент поверхностного натяжения нефти
        "muo": muo,                     #вязкость нефти
}

calibr_C_grav = 1                       #коэффициент калибровки гравитационной компоненты градиента
calibr_C_fric = 1                       #коэффициент калибровки фрикционной компоненты градиента

args_grav_grad = {
        "theta_deg": theta,
        "c_calibr_grav": calibr_C_grav,
}

args_fric_grad = {
        "eps_m": epsilon,
        "mug_rc_cp": mug,
        "c_calibr_fric": calibr_C_fric,
        "rho_lrc_kgm3": rho_lrc,
        "rho_grc_kgm3": rho_grc,
        "T" : T,
        "wc": wc,
        "sigmao": sigmao,
        "muo" : muo,
}

In [61]:
qo /sec_p_day / ql, ql / qg

(0.8366611530696679, 1.345825599526718)

In [62]:
# протестируем работоспособность класса DunsRos()
dr_corr = DR.DunsRos()
dr_corr.calc_params(**args_params)  #класс подразумевает предварительный рассчет параметров данным методом, иначе работать не будет
dr_corr.calc_grav(**args_grav_grad) + dr_corr.calc_fric(**args_fric_grad), dr_corr.calc_grav(**args_grav_grad), dr_corr.calc_fric(**args_fric_grad)

(4727.0176265215705, 4690.482685789894, 36.53494073167611)

In [63]:
dr_corr.hl, dr_corr.ll, dr_corr.hl > dr_corr.ll

(0.6363984376696892, 0.5737108503710783, True)

In [64]:
# создадим функцию для вычисления градиента
def calc_DP_DZ(
        T: float,                       #температура смеси
        d: float,                       #диаметр в метрах
        theta: float,                   #угол
        q0: float,                        
        delta_qg: float,
        rs: float,                        
        rho0: float,                    #плотность нефти
        rhog: float,                    #плотность газа
        muo: float,                       
        epsilon : float,                #шероховатость стенки трубы
        sigmao: float,                  #коэффициент поверхностного натяжения нефти
        wc: float,                      #коэффициент заводнени
        p: float,
):
    
    mug = 0.016 * 1e-3                                      #вязкость газа
    qo = q0 * (1 - wc) * bo                                 #объем нефти в пластовых условиях
    ql = ((1 - wc) * bo + wc * bw) * qo                     #объем жидкости в пластовых условиях
    qg = qo * rs * bg * (1 - wc) + delta_qg                 #объем газа в пластовых условиях
    rho_lrc = rho0 * (1 - wc) / bo + rhow * wc / bw         #плотность жидкой фазы
    rho_grc = rhog / rs                                     #плотность газа (?) 
    
    sec_p_day = 24 * 60 * 60                                #переведем все в систему СИ
    ql = ql / sec_p_day
    qg = qg  / sec_p_day
    
    args_params = {
        "d": d,                         #диаметр в метрах
        "theta_deg": theta,             #угол
        "ql_rc_m3day": ql,              #дебит жидкости
        "qg_rc_m3day": qg,              #расход газа
        "rho_lrc_kgm3": rho_lrc,        #плотность жидкой фазы
        "rho_grc_kgm3": rho_grc,        #плотность газа
        "p": p,                         #давление (где?)
        "T" : T,                        #температура смеси
        "wc": wc,                       #коэффициент заводнения
        "sigmao": sigmao,               #коэффициент поверхностного натяжения нефти
        "muo": muo,                     #вязкость нефти
        }
    
    calibr_C_grav = 1                   #коэффициент калибровки гравитационной компоненты градиента
    calibr_C_fric = 1                   #коэффициент калибровки фрикционной компоненты градиента
    
    args_grav_grad = {
        "theta_deg": theta,
        "c_calibr_grav": calibr_C_grav,
        }
    
    args_fric_grad = {
        "eps_m": epsilon,
        "mug_rc_cp": mug,
        "c_calibr_fric": calibr_C_fric,
        "rho_lrc_kgm3": rho_lrc,
        "rho_grc_kgm3": rho_grc,
        "T" : T,
        "wc": wc,
        "sigmao": sigmao,
        "muo" : muo,
        }
    
    dr_corr = DR.DunsRos()
    dr_corr.calc_params(**args_params)
    if dr_corr.hl > dr_corr.ll:
        DP_DZ = dr_corr.calc_grav(**args_grav_grad) + dr_corr.calc_fric(**args_fric_grad)
        return DP_DZ, dr_corr.fp
    else:
        return None


In [67]:
T = 50                          #температура гр.Ц.
d = 0.154                       #диаметр трубы
theta = 90                      #угол наклона скважины
q0 = 1000                       #дебит нефти н.у.
delta_qg = 1000
rs = 5                          #газовый фактор — коэффициент растворимости газа в нефти (конденсации) или коэффициент выделения газа из нефти (выкипания)
rho0 = 800                      #плотность нефти
rhog = 700                      #плотность газа
muo = 0.97 * 1e-3               #вязкость нефти
epsilon = 18.288 * 1e-6         #шероховатость стенки трубы
sigmao = 8.41 * 1e-3            #коэффициент поверхностного натяжения нефти
wc = 0.009

calc_DP_DZ(T, d, theta, q0, delta_qg, rs, rho0, rhog, muo, epsilon, sigmao, wc, 1)

(4727.0176265215705, 0)

In [68]:
count = 5
count_min = 3

# Набираем диапазоны выбранных переменных для перебора. Диапазоны короткие, т.к. при переборе будет count**13 значений (у нас 13 входных параметров)
T = np.linspace(10, 100, count_min)                    #температура гр.Ц.
d = np.linspace(0.11, 0.32, count)                     #диаметры труб
theta = np.linspace(0, 90, count_min)                  #уголы наклона
q0 = np.linspace(5, 1000, count)                       #дебит нефти п.у. м**3/с
delta_qg = np.linspace(0, 1000, count)                 #дельта к дебиту нефти, чтобы не получить отрицательные значения по расходу газа (для корректного рассчета коррелляции)
rs = np.linspace(5, 30, count)                         #коэффициент растворимости газа в нефти (конденсации) или коэффициент выделения газа из нефти (выкипания)
rho0 = np.linspace(780, 900, count_min)                #плотность нефти
rhog = np.linspace(700, 1300, count)                   #плотность газа
muo = np.linspace(1.5 * 1e-3, 150 * 1e-3, count)       #вязкость нефти
epsilon = np.linspace(15 * 1e-6, 300 * 1e-6, count)    #шероховатость стенки трубы
sigmao = np.linspace(10 * 1e-3, 30 * 1e-3, count_min)  #коэффициент поверхностного натяжения нефти
wc = np.linspace(0.05,0.98,count)                      #коэффициент обводненности
p = np.linspace(10,100,count)                          #давление (где?)

In [69]:
iters_count = 0
for T_i, d_i, theta_i, q0_i, delta_qg_i, rs_i, rho0_i, rhog_i, muo_i, epsilon_i, sigmao_i, wc_i in itertools.product(T, d, theta, q0, delta_qg, rs, rho0, rhog, muo, epsilon, sigmao, wc): iters_count += 1
iters_count

31640625

In [70]:
iters_count = 32822625
data_fp_0_1_dict = []
data_fp_2_3_dict = []
with tqdm(total=iters_count) as pbar:
    for T_i, d_i, theta_i, q0_i, delta_qg_i, rs_i, rho0_i, rhog_i, muo_i, epsilon_i, sigmao_i, wc_i in itertools.product(T, d, theta, q0, delta_qg, rs, rho0, rhog, muo, epsilon, sigmao, wc):
        pbar.update(1)
        DP_DZ, mode = calc_DP_DZ(T_i, d_i, theta_i, q0_i, delta_qg_i, rs_i, rho0_i, rhog_i, muo_i, epsilon_i, sigmao_i, wc_i, 1)
        if (DP_DZ != None) and (DP_DZ >= 0) and (mode == 0 or mode == 1):
            data_fp_0_1_dict.append({'T' : T_i, 
                                     'd' : d_i,
                                     'theta' : theta_i,
                                     'q0' : q0_i,
                                     'delta_qg' : delta_qg_i,
                                     'rs' : rs_i,
                                     'rho0' : rho0_i,
                                     'rhog' : rhog_i,
                                     'muo' : muo_i,
                                     'epsilon' : epsilon_i,
                                     'sigmao' : sigmao_i,
                                     'wc' : wc_i,
                                     'DP_DZ' : DP_DZ,
                                     'mode': mode,
                                    })
        elif (DP_DZ != None) and (DP_DZ >= 0) and (mode == 2 or mode == 3):
            for p_i in p:
                DP_DZ, mode = calc_DP_DZ(T_i, d_i, theta_i, q0_i, delta_qg_i, rs_i, rho0_i, rhog_i, muo_i, epsilon_i, sigmao_i, wc_i, p_i)
                data_fp_2_3_dict.append({'T' : T_i, 
                                        'd' : d_i,
                                        'theta' : theta_i,
                                        'q0' : q0_i,
                                        'delta_qg' : delta_qg_i,
                                        'rs' : rs_i,
                                        'rho0' : rho0_i,
                                        'rhog' : rhog_i,
                                        'muo' : muo_i,
                                        'epsilon' : epsilon_i,
                                        'sigmao' : sigmao_i,
                                        'wc' : wc_i,
                                        'p' : p_i,                                        
                                        'DP_DZ' : DP_DZ,
                                        'mode': mode,
                                        })


 96%|█████████▋| 31640625/32822625 [2:14:49<05:02, 3911.48it/s]  


In [72]:
data_0_1 = pd.DataFrame(data_fp_0_1_dict)
# data_2_3 = pd.DataFrame(data_fp_2_3_dict)
data_0_1.to_csv('data_0_1.csv')
# data_2_3.to_csv('data_2_3.csv')

In [79]:
data_0_1.head(10)

,T,d,theta,q0,delta_qg,rs,rho0,rhog,muo,epsilon,sigmao,wc,DP_DZ,mode
0,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.01,0.0500,0.003231,0
1,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.01,0.2825,0.001936,0
2,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.01,0.5150,0.000932,0
3,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.01,0.7475,0.000275,0
4,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.01,0.9800,0.000003,0
5,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.02,0.0500,0.003231,0
6,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.02,0.2825,0.001936,0
7,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.02,0.5150,0.000932,0
8,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.02,0.7475,0.000275,0
9,10.0,0.11,0.0,5.0,0.0,5.0,780.0,700.0,0.0015,0.000015,0.02,0.9800,0.000003,0


In [83]:
data_0_1['DP_DZ'].min(), data_0_1['DP_DZ'].max()

(1.724929631691617e-09, 9761.318266811797)

In [84]:
data_0_1[data_0_1['mode'] == 1].count()['mode'], data_0_1[data_0_1['mode'] == 0].count()['mode']

(9748225, 21596900)

In [ ]:
# let's construct a dataset
f = lambda x: x[:,0]**2 + 0.3*x[:,1] + 0.1*x[:,2]**3 + 0.0*x[:,3]
dataset = create_dataset(f, n_var=4, device=device)

input_vars = [r'$x_'+str(i)+'$' for i in range(4)]

model = KAN(width=[4,5,1], device=device)
model.fit(dataset, steps=40, lamb=0.001);

In [2]:
data = pd.read_csv('data_0_1.csv')

In [10]:
data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31345125 entries, 0 to 31345124
Data columns (total 15 columns):
 #   Column      Non-Null Count     Dtype  
---  ------      --------------     -----  
 0   Unnamed: 0  31345125 non-null  int64  
 1   T           31345125 non-null  float64
 2   d           31345125 non-null  float64
 3   theta       31345125 non-null  float64
 4   q0          31345125 non-null  float64
 5   delta_qg    31345125 non-null  float64
 6   rs          31345125 non-null  float64
 7   rho0        31345125 non-null  float64
 8   rhog        31345125 non-null  float64
 9   muo         31345125 non-null  float64
 10  epsilon     31345125 non-null  float64
 11  sigmao      31345125 non-null  float64
 12  wc          31345125 non-null  float64
 13  DP_DZ       31345125 non-null  float64
 14  mode        31345125 non-null  int64  
dtypes: float64(13), int64(2)
memory usage: 3.5 GB


In [11]:
len(data.index)-len(data.drop_duplicates().index)

0

In [12]:
data.isna().sum()

Unnamed: 0    0
T             0
d             0
theta         0
q0            0
delta_qg      0
rs            0
rho0          0
rhog          0
muo           0
epsilon       0
sigmao        0
wc            0
DP_DZ         0
mode          0
dtype: int64

In [18]:
data[data['DP_DZ'] == 0].count()['DP_DZ']

0